In [2]:
import os
import openai
from dotenv import load_dotenv, find_dotenv
import pandas as pd
import numpy as np
import time

In [3]:
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']

llm_model = "gpt-3.5-turbo-16k"
llm_embedding = "text-embedding-ada-002" # simple: pasar todos las frases vectores y replicar el mismo procedimiento.

# Opciones para el uso de memoria

* `ConversationBufferWindowMemory`: recuerda una cantidad de inputs y outputs
* `ConversationTokenBufferMemory`: recuerda una cantidad de tokens, esto puede cortar la conversación
* `ConversationSummaryBufferMemory`: esto hace un resumen de la conversación 


Tenemos un limite de 2048 tokens de input... sacar una media de tokens necesarios.

In [3]:
# Data import
# Relative Path of the dataset, change for your dataset
dataset_name = "cpn27"
file_path = '../data/raw_dataset/'

if dataset_name == "cpn27":
    data = pd.read_csv(file_path+'CPN27.csv', delimiter=",")
    
elif dataset_name == "cpn120":
    data = pd.read_csv(file_path+'CPN120.csv', delimiter=",")


data.fillna(value='')

,Palabra (concepto),Descripción,Codificación
0,Compasión,sentimiento,sentimiento
1,Compasión,por lo que se le perdona la vida a alguien en ...,perdón
2,plan,necesario para cumplir objetivos,objetivos
3,plan,organizar recursos o personas,organización
4,plan,estrategia,estrategia
...,...,...,...
4933,Obligación,imposicion,imposición
4934,Obligación,carácter extricto,extricto
4935,Obligación,normas,normas
4936,Obligación,deberes,deber


In [7]:
from sklearn.model_selection import StratifiedKFold

kf = StratifiedKFold(n_splits = 5)

X = data.iloc[:,:2]
y = data.iloc[:,2]

for i, (train_index, test_index) in enumerate(kf.split(X, y)):
    data_test = data.iloc[test_index,1:].reset_index(drop=True)

    prev_num = 0 
    for j in np.linspace(120, data_test.shape[0], num=8): 
        
        data_test.iloc[prev_num:int(j),:].to_csv(r'ChatGPT_inputs/input_fold{i}_from{prev_num}to{j}.csv'.format(i=i, prev_num=prev_num, j=int(j)),header=False, index=False)
        
        prev_num = int(j)

    

c:\Users\dra98\OneDrive\Documentos\Trabajo\Doctorado\Codigo\myvenv\lib\site-packages\sklearn\model_selection\_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


In [6]:
import glob

test_files = []

for i in range(5):
    folder = glob.glob(f'ChatGPT_inputs/input_fold{i}_*')
    test_files.append(folder)

values_completed = []

In [7]:
from langchain.chains import LLMChain
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

# Chat GPT model
llm = ChatOpenAI(temperature=0, model=llm_model)


prompt_raw = '''Respira hondo y trabaja en este problema paso a paso.\
 Para las siguientes propiedades genera un código de una palabra, un código es una palabra\
 real que representa el significado de una propiedad, como por ejemplo las palabras\
 \'cuatro patas\' y \'cuadrúpedo\' son propiedades que se refieren a lo mismo que es el código de: \'cuadrúpedo\'.
Entrega el resutlado con notación de CSV: 

\'propiedad\', \'código\',

Queremos poder unir las propiedades que sean similares en significado,\
 de la misma forma que puede haber una codificación única, se pueden\
 repetir entre propiedades, pero solo pueden tener un código.\
 Codifica para las siguientes propiedades:

{examples}'''


examples = '''
PROP: {property},
CODE:

'''

# LLM chain
first_prompt = ChatPromptTemplate.from_template(prompt_raw)
chain_codification = LLMChain(llm=llm, prompt=first_prompt, output_key='codes')



for i, files in enumerate(test_files):
    for j, f in enumerate(files):
        
        tempdf = pd.read_csv(f)

        input = '\n'.join(
            [examples.format(property=i) for i in tempdf.iloc[:,1]]
            )
        # Llamado a ChatGPT, se demora aproximadamente 5 min cada llamado
        results = chain_codification.run(input)
        
        with open(f"ChatGPT_outputs/output_fold{i}_{j}.csv", "w") as text_file:
            text_file.write(results)
            text_file.close
        # break
        values_completed.append((i, j))
        print(i, j)
        time.sleep(3600)



0 0


In [ ]:
results
with open("Output.csv", "w") as text_file:
    text_file.write(results)
    text_file.close

In [ ]:
temp_y = y[test_index].reset_index(drop=True)
acc = 0
for i, property in enumerate(X_test.iloc[:200,1]):
    codification = g_codes[g_codes[:,0] == property, 1]
    if len(g_codes[g_codes[:,0] == property, 1]) == 0:
        pass
    elif len(g_codes[g_codes[:,0] == property, 1]) == 1:
        if codification[0] == temp_y[i]:
            print(codification[0])
            acc += 1
        else:
            print(codification, "vs", temp_y[i])
    elif len(g_codes[g_codes[:,0] == property, 1]) > 1:
        if codification[0] == temp_y[i]:
            print(codification[0])
            acc += 1
print(acc/200)

['emoción'] vs sentimiento
['clemencia'] vs perdón
['requisito'] vs objetivos
['gestión'] vs organización
['habilidad'] vs capacidad
['personalidad'] vs personal
['eficacia'] vs meta
['materiales'] vs blandas
['producción'] vs trabajo
['beneficio'] vs utilidad
['verdad'] vs no mentir
['esperanza'] vs fe
['logro'] vs meta
['victoria'] vs ganar
['creatividad'] vs innovación
['rentabilidad'] vs éxito
['felicidad'] vs alegría
['emoción'] vs sentimiento
['emoción'] vs sentimiento
['entendimiento'] vs comprensión
['emoción'] vs sentimiento
['razonamiento'] vs lógica
['examen'] vs análisis
['existencia'] vs vida
['concepto'] vs abstracción
['tolerancia'] vs paciencia
['intranquilidad'] vs impaciencia
['irreflexivo'] vs impulso
['indefinición'] vs ambigüedad
['prisa'] vs apuro
alegría
['confort'] vs comodidad
['equilibrio'] vs estabilidad
['balance'] vs equilibrio
['dominio'] vs control
['específico'] vs concreto
['realizado'] vs hechos
['permanente'] vs constante
['exactitud'] vs precisión
['

In [ ]:
from sklearn.model_selection import StratifiedKFold
from langchain.chains import LLMChain
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
import re

response = np.zeros(5, dtype=str)

kf = StratifiedKFold(n_splits = 5)

X = data.iloc[:,:2]
y = data.iloc[:,2]


# Chat GPT model
llm = ChatOpenAI(temperature=0, model=llm_model)


prompt_raw = '''Respira hondo y trabaja en este problema paso a paso.\
 Para las siguientes propiedades genera un código de una palabra, un código es una palabra\
 real que representa el significado de una propiedad, como por ejemplo las palabras\
 \'cuatro patas\' y \'cuadrúpedo\' son propiedades que se refieren a lo mismo que es el código de: \'cuadrúpedo\'.
Entrega el resutlado con notación de CSV: 

\'propiedad\', \'código\',

Queremos poder unir las propiedades que sean similares en significado,\
 de la misma forma que puede haber una codificación única, se pueden\
 repetir entre propiedades, pero solo pueden tener un código.\
 Codifica para las siguientes propiedades:

{examples}'''


examples = '''
PROP: {property},
CODE:

'''

# LLM chain
first_prompt = ChatPromptTemplate.from_template(prompt_raw)
chain_codification = LLMChain(llm=llm, prompt=first_prompt, output_key='codes')

acc_gpt = []

for i, (train_index, test_index) in enumerate(kf.split(X,y)):
    # X_train = X.iloc[train_index,:] 
    X_test = X.iloc[test_index,:]
    # y_train = y[train_index].reset_index(drop=True)
    y_test = y[test_index].reset_index(drop=True)
    # break


    prev_num = 0
    acc = 0
    for i in np.linspace(120, X_test.shape[0], num=8): 
        
        # Prompt con las codificaciones
        # Tiene un maximo de 200 propiedades que funciona sin problemas
        input = '\n'.join(
            [examples.format(property=i) for i in X_test.iloc[prev_num:int(i),1]]
            )
        # Llamado a ChatGPT, se demora aproximadamente 5 min cada llamado
        results = chain_codification.run(input)
        print('promt complete')

        answers = results.split('\'\'\'')

        regex = r"\n(\w+[\s*\w*]+)\nCODE: (\w+)|\n'(\w+[\s*\w*]+)'\nCODE: '(\w+)'|\n(\w+[\s*\w*]+)\nCODE: '(\w+)'"
        g_codes = []


        for a in answers:
            matches = re.finditer(regex, a)
            for matchNum, match in enumerate(matches, start=1):
                if len(match.groups())>0:
                    temp = []
                    for groupNum in range(0, len(match.groups())):
                        groupNum = groupNum + 1
                        temp.append(match.group(groupNum))
                    g_codes.append(temp)


        g_codes = np.array(g_codes)
        break
        
        
        for i, property in enumerate(X_test.iloc[:200,1]):
            codification = g_codes[g_codes[:,0] == property, 1]
            if len(g_codes[g_codes[:,0] == property, 1]) == 0:
                pass
            elif len(g_codes[g_codes[:,0] == property, 1]) == 1:
                if codification[0] == y_test[i]:
                    acc += 1
            elif len(g_codes[g_codes[:,0] == property, 1]) > 1:
                if codification[0] == y_test[i]:
                    acc += 1
    
        prev_num = int(i)
    break
    acc_gpt.append(acc/len(y_test))

    
    

    

c:\Users\dra98\OneDrive\Documentos\Trabajo\Doctorado\Codigo\myvenv\lib\site-packages\sklearn\model_selection\_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


promt complete
